Multi-resolution registration example

# Working with big data

## Strategy 1 - downsample

Registration algorithms can be expensive, but will be much faster if working on lower resolution images. In this notebook, we will 

1. run a registration algorithm on the downsampled images we produced in `resample_images.ipynb`
2. apply the transformation that results on the original, high-resolution images.

Two facts make this possible.

* the original and downsampled images are in the same **physical** coordinate system.
* the transformation produced by teh registration works on **physical** (not pixel) coordinates.

First, let's import libraries and make some useful functions:

In [ ]:
import os
from itertools import chain
import numpy as np

import SimpleITK as sitk

def run_elastix(fixed_image, moving_image, params):
    """
    Runs elastix with the given fixed and moving images,
    and elastix parameters.
    
    Returns the ElastixImageFilter, from which the results
    can be obtained.
    """
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

def print_parameters(elastix_parameters):
    for k,v in elastix_parameters.iteritems():
        print(f'{k} \n\t {v}')

Next, we'll load the downsampled images.

In [ ]:
# load fixed image
fixed_image_path='../sampleImages/jrc18_down.nrrd'
fixed_image = sitk.ReadImage(fixed_image_path)

# load moving image
moving_image_path = '../sampleImages/jrc10_down.nrrd'
moving_image = sitk.ReadImage(moving_image_path)

In [ ]:
%%time

# run a simple registration
affine_params = sitk.ReadParameterFile('../elastixParameters/DefaultSmoother_Affine.txt')
elastixImageFilter = run_elastix(fixed_image, moving_image, affine_params ) 

# write the output to a file
sitk.WriteImage(elastixImageFilter.GetResultImage(), '../sampleImages/affine_result_img.nrrd')


In [ ]:
# print the output
transform_parameters = elastixImageFilter.GetTransformParameterMap()
print_parameters(transform_parameters[0])

This output contains lots of information, but notice three things in particular:

1. The `Size` field tells elastix what the pixel size of the output image 
2. The `Spacing` field tells elastix the resolution of the 
3. The `TransformParameters` store the affine transformation that will be applied to the image.

Next, we'll apply this transformation to the high resolution image, so the first step is to load it.

In [ ]:
moving_image_hi_path = '../sampleImages/JFRCtemplate2010.nrrd'
moving_image_hi = sitk.ReadImage(moving_image_hi_path)
print( moving_image_hi.GetSpacing() )

Finally, we'll apply the transformation to this higher resolution image.

Elastix's default behavior is to transform images to match the size and resolution of the fixed image. In this case, our downsampled fixed image has resolution `1.52 x 1.52 x 1.52`, and its size is `413 x 192 x 120`, so that is the size and resolution of the output we would get.

What we want is an output image matching the original high-resolution fixed image, which has resolution `0.38 x 0.38 x 0.38` and size `1652 x 768 x 479`. We need to tell elastix that this is what we want, so we set the appropriate values of the transform parameters, like so:

In [ ]:
transform_parameters_highres = elastixImageFilter.GetTransformParameterMap()
transform_parameters_highres[0]['Size'] = ['1652', '768', '479']
transform_parameters_highres[0]['Spacing'] = ['0.38', '0.38', '0.38']

and now we can use those edited parameters to apply the transformation.

In [ ]:
# "Transformix" is the filter that comes with elastix that transforms images  
transformixImageFilter = sitk.TransformixImageFilter()

# set the transformation and moving image, and run 
transformixImageFilter.SetTransformParameterMap(transform_parameters_highres)
transformixImageFilter.SetMovingImage(moving_image_hi)

# run it
transformixImageFilter.Execute()

# write the resulting image
sitk.WriteImage(transformixImageFilter.GetResultImage(), '../sampleImages/affine_result_hires_img.nrrd')

 ## Check the results
 
 Use Fiji to open the resulting image, located in `sampleImages/affine_result_hires_img.nrrd`. Confirm that:
 
 * It is the expected size and resolution
 * It is well registered to the high-resolution target image

 ## Bonus (optional)
 
Run transformix again using the higher resolution moving image, but this time use transform parameters directly from elastix `transform_parameters = elastixImageFilter.GetTransformParameterMap()`.  (i.e., don't set the output size and resolution).  Confirm that the resulting image:

* Is the expected, smaller size and resolution
* Is still well registered to the fixed image.


## Advanced (optional)

Run transformix again with the high resolution image, but this time create an image 
at resolution `[0.5, 0.5, 0.75]`.

1. determine a reasonable pixel size for the resulting image.
2. run `transform_parameters = elastixImageFilter.GetTransformParameterMap()`
2. set the values for the `transform_parameters` appropriately.
3. run transformix
4. verify that the result is as expected
